In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, InputLayer
from keras.optimizers import Adam

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy as cp
from sklearn.metrics import r2_score

print(f'tf_version: {tf.__version__}')

2023-08-21 18:29:18.185567: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-21 18:29:19.022681: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-21 18:29:19.025421: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 18:29:22.360592: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


tf_version: 2.13.0


## Data Analysis

In [2]:
# Loading the dataset
original_df = pd.read_csv('./data/CaliforniaHousing.csv', sep=',')
df = cp.deepcopy(original_df)
display(df.head())

target = 'price'
features = [i for i in df.columns if i not in target]

x_train_df = df[['area', 'bedrooms', 'bathrooms', 'stories', 'parking']]
x_train_np = x_train_df.to_numpy()
y_train_np = df[target].to_numpy().reshape(-1,1)
print(f'x_train shape: {x_train_np.shape} \t y_train shape: {y_train_np.shape}')

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


x_train shape: (545, 5) 	 y_train shape: (545, 1)


## Data Preprocessing

### Feature Scalling
In order to make the training algorithm faster, we normalize the inputs using Keras Normalization layer

In [3]:
print('Before Normalization:')
print(f'Peak-to-peak x_train: {np.ptp(x_train_np, axis=0)}')
display(pd.DataFrame(x_train_np, columns=x_train_df.columns).describe())

# Creating the normalization layer
norm_1 = tf.keras.layers.Normalization(axis=-1) 
# Does the same as passing mean and variance in layer construction
norm_1.adapt(x_train_np) 

# Foward propagating x_train
x_train_norm = norm_1(x_train_np)

print('\nAfter Normalization:')
print(f'Peak-to-peak x_train: {np.ptp(x_train_norm, axis=0)}')
display(pd.DataFrame(x_train_norm, columns=x_train_df.columns).describe())

Before Normalization:
Peak-to-peak x_train: [14550     5     3     3     3]


,area,bedrooms,bathrooms,stories,parking
count,545.000000,545.000000,545.000000,545.000000,545.000000
mean,5150.541284,2.965138,1.286239,1.805505,0.693578
std,2170.141023,0.738064,0.502470,0.867492,0.861586
min,1650.000000,1.000000,1.000000,1.000000,0.000000
25%,3600.000000,2.000000,1.000000,1.000000,0.000000
50%,4600.000000,3.000000,1.000000,2.000000,0.000000
75%,6360.000000,3.000000,2.000000,2.000000,1.000000
max,16200.000000,6.000000,4.000000,4.000000,3.000000



After Normalization:
Peak-to-peak x_train: [6.7107935 6.7807055 5.9759965 3.4614208 3.4851508]


,area,bedrooms,bathrooms,stories,parking
count,5.450000e+02,5.450000e+02,5.450000e+02,5.450000e+02,5.450000e+02
mean,-8.399333e-08,-1.259900e-07,1.959844e-07,1.539878e-07,-2.799778e-08
std,1.000919e+00,1.000919e+00,1.000919e+00,1.000919e+00,1.000919e+00
min,-1.614530e+00,-2.665004e+00,-5.701866e-01,-9.293966e-01,-8.057413e-01
25%,-7.151452e-01,-1.308863e+00,-5.701866e-01,-9.293966e-01,-8.057413e-01
50%,-2.539223e-01,4.727817e-02,-5.701866e-01,2.244103e-01,-8.057413e-01
75%,5.578299e-01,4.727817e-02,1.421812e+00,2.244103e-01,3.559756e-01
max,5.096263e+00,4.115701e+00,5.405810e+00,2.532024e+00,2.679410e+00


### Tiling

Tile/copy our data to increase the training set size and reduce the number of training epochs.

In [4]:
x_train = np.tile(x_train_norm, (100, 1)) # Repeat 1000 times the rows while keeping the columns.
y_train = np.tile(y_train_np, (100, 1))

print(f'x_train shape before: {x_train_norm.shape}')
print(f'x_train shape after: {x_train.shape}')

x_train shape before: (545, 5)
x_train shape after: (54500, 5)


## Model Training

### Perceptron Training

![Alt text](images/perceptron.png)

The function implemented by a neuron with no activation is the same as linear regression:

![Alt text](images/linearfunction.png)

In [7]:
tf.random.set_seed(100) # applied to achieve consistent results 

# Creates a linear neural network with 3 hidden layers 
L_0 = InputLayer(input_shape=[x_train.shape[1],]) # This layer can be omitted 
L_1 = Dense(units=1, activation='linear', name='layer_1')

nn_linear = Sequential([L_0, L_1], name='Linear_Model')

display(nn_linear.summary()) # The parameter counts correspond to the number of elements in the weight and bias.

W1, b1 = nn_linear.get_layer("layer_1").get_weights()
print('Initialized weights: \n')
print(f"W1{W1.shape}:\n", W1, f"\nb1{b1.shape}:", b1, '\n')

Model: "Linear_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_1 (Dense)             (None, 1)                 6         
                                                                 
Total params: 6 (24.00 Byte)
Trainable params: 6 (24.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


None

Initialized weights: 

W1(5, 1):
 [[-0.93486285]
 [ 0.23711252]
 [-0.58544755]
 [-0.30329227]
 [-0.6100495 ]] 
b1(1,): [0.] 



In [8]:
# Defines loss function and optimizer
nn_linear.compile( 
    loss = 'mean_squared_error',
    optimizer = Adam(learning_rate=2000),
)

# Runs gradient descent and fits the weights to the data
nn_linear.fit(
    x_train,y_train,            
    epochs=20,
)

# Learning_rate had to be set to 2000 due to slow conversion in the first tests

Epoch 1/20
1704/1704 [==============================] - 5s 2ms/step - loss: 13031592427520.0000
Epoch 2/20
1704/1704 [==============================] - 2s 1ms/step - loss: 2826081665024.0000
Epoch 3/20
1704/1704 [==============================] - 2s 1ms/step - loss: 1557503803392.0000
Epoch 4/20
1704/1704 [==============================] - 2s 1ms/step - loss: 1532513615872.0000
Epoch 5/20
1704/1704 [==============================] - 2s 1ms/step - loss: 1532240592896.0000
Epoch 6/20
1704/1704 [==============================] - 2s 1ms/step - loss: 1532969615360.0000
Epoch 7/20
1704/1704 [==============================] - 2s 1ms/step - loss: 1532331950080.0000
Epoch 8/20
1704/1704 [==============================] - 2s 1ms/step - loss: 1532750069760.0000
Epoch 9/20
1704/1704 [==============================] - 2s 1ms/step - loss: 1532550578176.0000
Epoch 10/20
1704/1704 [==============================] - 2s 1ms/step - loss: 1532750462976.0000
Epoch 11/20
1704/1704 [=========================

Comparisson with the linear regression model:

![Alt text](images/comparissonLinearNN.png)

In [9]:
print('Found w, b: ', nn_linear.get_layer('layer_1').get_weights())

Found w, b:  [array([[726110.75],
       [140668.27],
       [576362.2 ],
       [478325.6 ],
       [333400.56]], dtype=float32), array([4779047.], dtype=float32)]


In [10]:
p_linear = nn_linear.predict(x_train_norm)
print('Prediction on training set: \n', p_linear[:5,:], '\n')
print('Target values: \n', y_train_np[:5,:], '\n')
print('Model accuracy (delta <= 1e6): %f'%(np.mean(np.abs(p_linear - y_train_np) <= 1000000)))
print(f'R2 Score: {r2_score(y_train_np, p_linear)}\n')

18/18 [==============================] - 0s 1ms/step
Prediction on training set: 
 [[ 7721217.5]
 [11472400. ]
 [ 7829199. ]
 [ 7583431. ]
 [ 6021209.5]] 

Target values: 
 [[13300000]
 [12250000]
 [12250000]
 [12215000]
 [11410000]] 

Model accuracy (delta <= 1e6): 0.653211
R2 Score: 0.5613030382491717



### Deep Feed Foward Model Training

![Alt text](images/dff.png)

In [76]:
# Creates a linear neural network with 3 hidden layers 
L_0 = InputLayer(input_shape=(x_train.shape[1],)) # This layer can be omitted 
L_1 = Dense(units=160, kernel_initializer='normal', activation='sigmoid', name='layer_1')
L_2 = Dense(units=480, kernel_initializer='normal', activation='sigmoid', name='layer_2')
L_3 = Dense(units=256, kernel_initializer='normal', activation='sigmoid', name='layer_3')
L_4 = Dense(units=1, kernel_initializer='normal', activation='linear', name='layer_4')

nn_dff = Sequential([L_0, L_1, L_2, L_3, L_4], name='DFF')

display(nn_dff.summary()) # The parameter counts correspond to the number of elements in the weight and bias.

Model: "DFF"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_1 (Dense)             (None, 160)               960       
                                                                 
 layer_2 (Dense)             (None, 480)               77280     
                                                                 
 layer_3 (Dense)             (None, 256)               123136    
                                                                 
 layer_4 (Dense)             (None, 1)                 257       
                                                                 
Total params: 201633 (787.63 KB)
Trainable params: 201633 (787.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


None

In [81]:
# Defines loss function and optimizer
nn_dff.compile( 
    loss = 'mse', # Mean squared error
    optimizer = Adam(learning_rate=1000),
)

# Runs gradient descent and fits the weights to the data
history = nn_dff.fit(
    x_train,y_train,            
    epochs=10,
    batch_size=64,
)

Epoch 1/10
852/852 [==============================] - 5s 5ms/step - loss: 3667590905856.0000
Epoch 2/10
852/852 [==============================] - 4s 5ms/step - loss: 3503156363264.0000
Epoch 3/10
852/852 [==============================] - 4s 5ms/step - loss: 3502934851584.0000
Epoch 4/10
852/852 [==============================] - 4s 5ms/step - loss: 3504568270848.0000
Epoch 5/10
852/852 [==============================] - 4s 5ms/step - loss: 3506805407744.0000
Epoch 6/10
852/852 [==============================] - 4s 5ms/step - loss: 3505184833536.0000
Epoch 7/10
852/852 [==============================] - 4s 5ms/step - loss: 3507026657280.0000
Epoch 8/10
852/852 [==============================] - 5s 6ms/step - loss: 3507702726656.0000
Epoch 9/10
852/852 [==============================] - 5s 6ms/step - loss: 3502275821568.0000
Epoch 10/10
852/852 [==============================] - 5s 6ms/step - loss: 3506691899392.0000


In [82]:
p_dff = nn_dff.predict(x_train_norm)
print(nn_dff.get_layer('layer_4').get_weights()[0].shape)
print('Prediction on training set: \n', p_dff[:5,:], '\n')
print('Target values: \n', y_train_np[:5,:], '\n')
print('Model accuracy (delta <= 1e6): %f'%(np.mean(np.abs(p_dff - y_train_np) <= 1000000)))
print(f'R2 Score: {r2_score(y_train_np, p_dff)}\n')

18/18 [==============================] - 0s 3ms/step
(256, 1)
Prediction on training set: 
 [[4868893.]
 [4868893.]
 [4868893.]
 [4868893.]
 [4868893.]] 

Target values: 
 [[13300000]
 [12250000]
 [12250000]
 [12215000]
 [11410000]] 

Model accuracy (delta <= 1e6): 0.385321
R2 Score: -0.002988848411113798



### Conclusion:
The DFF model took much more time to converge to a minimum loss in comparisson to the linear regressor model and performed poorly. The prediction for any input is the same and equal to the mean of the y_train values. This probably means that the dataset is more suitable for a linear model than ANN and I coudn't create a better model using ANN for this dataset.

PS. The training didn't use all the avaliable features.